In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from my_utils import seq_10, DataLoader
from torch_models.utils import seq2label

device = 'cpu'
trans_func = seq2label(device)
train = seq_10(5000)
test = seq_10(500)
train_loader = DataLoader(train, batch_size=16, trans_func=trans_func)
test_loader = DataLoader(test, batch_size=64, trans_func=trans_func)

In [12]:
from torch_models.models import BoV, RNNLastHidden, RNNMaxPool, SingleClassifier
from torch_models.utils import get_device


dropout = 0
encoder = RNNMaxPool(embed_size=10, hidden_size=15, vocab_size=10, bidirectional='cat',
                        num_layers=1, dropout=dropout, rnn='gru')
model = SingleClassifier(encoder=encoder, output_size=2, hidden_size=None,
                         activation='Tanh', dropout=dropout, freeze_encoder=False)
print(model)

SingleClassifier(
  (encoder): RNNMaxPool(
    (embedding): Embedding(11, 10, padding_idx=10)
    (rnn): GRU(10, 15, bidirectional=True)
  )
  (mlp): MLP(
    (fc_out): Linear(in_features=30, out_features=2, bias=True)
    (dropout): Dropout(p=0)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
)


In [13]:
from my_utils import Trainer, EvaluatorC, ScoreMonitor, EvaluatorLoss
from torch.optim import Adam

optimizer = Adam(model.parameters())

evaluator = EvaluatorC(model, test_loader)
score_monitor = ScoreMonitor(threshold=1, go_up=True)

trainer = Trainer(model, train_loader)
trainer.train_epoch(optimizer, max_epoch=5,
              evaluator=evaluator, score_monitor=score_monitor)

[2018-10-15 16:47:23,694 INFO] epoch [1/5]	loss: 0.45638031913592414	
[2018-10-15 16:47:23,739 INFO] Evaluator accuracy: 0.924	
[2018-10-15 16:47:25,563 INFO] epoch [2/5]	loss: 0.11841731336408148	
[2018-10-15 16:47:25,601 INFO] Evaluator accuracy: 0.998	
[2018-10-15 16:47:27,633 INFO] epoch [3/5]	loss: 0.039588210740150474	
[2018-10-15 16:47:27,671 INFO] Evaluator accuracy: 1.0	
[2018-10-15 16:47:30,039 INFO] epoch [4/5]	loss: 0.016530916707727095	
[2018-10-15 16:47:30,085 INFO] Evaluator accuracy: 1.0	
[2018-10-15 16:47:30,086 INFO] Dev score saturated.


In [6]:
trainer.train_step(optimizer, checkpoint_steps=1000, max_steps=5000,
              evaluator=evaluator, score_monitor=score_monitor)

[2018-10-15 11:45:32,391 INFO] Start Training
[2018-10-15 11:45:37,379 INFO] steps [3000/5000]	loss: 0.0008452482784960012	
[2018-10-15 11:45:37,410 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:45:37,410 INFO] Dev score saturated.
[2018-10-15 11:45:37,412 INFO] Stop Training


In [9]:
from my_utils import Trainer, EvaluatorC, ScoreMonitor
from torch.optim import Adam, SGD

lr = 3
lr_decay = 0.5
lr_threshold = 0.1
optimizer = SGD(model.parameters(), lr=3)
evaluator = EvaluatorC(model, test_loader)
score_monitor = ScoreMonitor(threshold=1)
trainer = Trainer(model, train_loader)


while lr > lr_threshold:
    print('lr: {}'.format(lr))
    trainer.train_epoch(optimizer, max_epoch=5,
                  evaluator=evaluator, score_monitor=score_monitor)
    lr *= lr_decay
    for param_group in optimizer.param_groups:
            param_group['lr'] = lr

[2018-10-15 11:51:34,750 INFO] Start Training


lr: 3


[2018-10-15 11:51:36,421 INFO] epoch [1/5]	loss: 0.0001610456483555468	
[2018-10-15 11:51:36,452 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:37,999 INFO] epoch [2/5]	loss: 0.00012702342168756096	
[2018-10-15 11:51:38,029 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:38,030 INFO] Dev score saturated.
[2018-10-15 11:51:38,031 INFO] Stop Training
[2018-10-15 11:51:38,032 INFO] Start Training


lr: 1.5


[2018-10-15 11:51:39,596 INFO] epoch [1/5]	loss: 0.00010839436630901449	
[2018-10-15 11:51:39,628 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:39,629 INFO] Dev score saturated.
[2018-10-15 11:51:39,630 INFO] Stop Training
[2018-10-15 11:51:39,632 INFO] Start Training


...

[2018-10-15 11:51:41,137 INFO] epoch [1/5]	loss: 0.00010039014712808208	
[2018-10-15 11:51:41,165 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:41,166 INFO] Dev score saturated.
[2018-10-15 11:51:41,167 INFO] Stop Training
[2018-10-15 11:51:41,169 INFO] Start Training


lr: 0.375


[2018-10-15 11:51:42,672 INFO] epoch [1/5]	loss: 9.713318516219624e-05	
[2018-10-15 11:51:42,699 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:42,700 INFO] Dev score saturated.
[2018-10-15 11:51:42,701 INFO] Stop Training
[2018-10-15 11:51:42,703 INFO] Start Training


lr: 0.1875


[2018-10-15 11:51:44,229 INFO] epoch [1/5]	loss: 9.539284052390922e-05	
[2018-10-15 11:51:44,259 INFO] Evaluator accuracy: 1.0	
[2018-10-15 11:51:44,260 INFO] Dev score saturated.
[2018-10-15 11:51:44,261 INFO] Stop Training
